![](https://raw.githubusercontent.com/wilocw/co2114-codebase/2024/static/0/uol_banner_red.png)

# CO2114<br />Foundations in Artificial Intelligence

# Tutorial 5 - AC-3

In this tutorial, we will work through an example of the AC-3 algorithm, a method for acheiving arc consistency in a constraint satisfaction problem.

## Setup
> **Run the following cell only if you are using Google Colab**

In [ ]:
!pip install "git+https://github.com/wilocw/co2114-codebase.git@2024#egg=agent&subdirectory=lab/src/2"
!pip install "git+https://github.com/wilocw/co2114-codebase.git@2024#egg=search&subdirectory=lab/src/3"
!pip install "git+https://github.com/wilocw/co2114-codebase.git@2024#egg=optimisation&subdirectory=lab/src/4"
!pip install "git+https://github.com/wilocw/co2114-codebase.git@2024#egg=constraints&subdirectory=lab/src/5"

## Imports

In [ ]:
from constraints.csp import ConstraintSatisfactionProblem, Variable, Factor

### `csp.Variable`

The `Variable` class is a data structure implemented to represent a variable in a constraint satisfaction problem.

#### Constructor: 
```python
A = Variable(domain, name=None)
```

#### Attributes:
- value: `A.value`
- domain: `A.domain`
- name [optional]: `A.name`
- is assigned: `A.is_assigned`

In [ ]:
domain = {0, 1, 2}  # example domain
A = Variable(domain, name="A")
A  # has no value

In [ ]:
A.is_assigned

In [ ]:
A.value = 1
A

In [ ]:
A.is_assigned

In [ ]:
try:
    A.value = 4  # 4 is not in domain
except Exception as e:
    print(e)

### `csp.Factor`

The `Factor` class is the data structure representing a factorised constraint: a constraint and the variables as part of it.

#### Constructor
```python
f = Factor(constraint, variables)
```

#### Attributes:
- variables: `f.variables`
- is satisfied: `f.is_satisfied` 
- is unary: `f.is_unary`
- is binary: `f.is_binary`
- arcs (binary only): `f.arcs`
- is global: `f.is_global`

It is also possible to evaluate a factor manually using `f(*variables)`. You also use the `in` keyword, to either check if a variable is in a factor, or iterate through variables, e.g.
```python
A in f  # true or false
```
or
```python
for variable in f:
    ...
```

##### Unary Constraint

A unary constraint is one that involves only one variable

In [ ]:
constraint = lambda a: a >= 2
f = Factor(constraint, A)
f

In [ ]:
print(f"{A.name}: {A}")
f.is_satisfied

In [ ]:
f.is_unary

##### Binary Constraint
A binary constraint involves exactly two variables

In [ ]:
B = Variable(domain, 'B')
print(f"{A.name}: {A}, {B.name}: {B}")

constraint = lambda a,b: a != b
g = Factor(constraint, (A, B))
g

In [ ]:
g.is_satisfied

In [ ]:
print(f"A in f: {A in f}")  # f is constraint on A?
print(f"B in f: {B in f}")  # f is constraint on B?
print(f"A in g: {A in g}")  # g is constraint on A?
print(f"B in g: {B in g}")  # g is constraint on B?

In [ ]:
g.is_unary

In [ ]:
g.is_binary

> All factors return True if any variable is unassigned

In [ ]:
constraint = lambda a,b: a==b  # the opposite of g
h = Factor(constraint, (A, B))
h

In [ ]:
print(f"{A.name}: {A}, {B.name}: {B}")
print(f"g is satisfied: {g.is_satisfied}\nh is satisfied: {h.is_satisfied}")

In [ ]:
B.value = 2
print(f"{A.name}: {A}, {B.name}: {B}")
h.is_satisfied

##### Global Constraint

A global constraint involves more than two variables

In [ ]:
C = Variable(domain, 'C')
print(f"{C.name}.domain={C.domain}")

> `alldiff` is common constraint that takes N variables, and returns True if all N are different

In [ ]:
from constraints.csp.util import alldiff

p = Factor(alldiff, (A, B, C))
p

In [ ]:
print(f"{A.name}: {A}, {B.name}: {B}, {C.name}: {C}")
p.is_satisfied

In [ ]:
C.value = 2
print(f"{A.name}: {A}, {B.name}: {B}, {C.name}: {C}")
p.is_satisfied  # ! 

#### `csp.ConstraintSatisfactionProblem`

A `ConstraintSatisfactionProblem` is simply the data structure that defines a CSP, defined by the variables, their domains and constraints.

#### Constructor
```python
csp = ConstraintSatisfactionProblem(variables, constraints)
```

#### Attributes:
- variables: `csp.variables`
- domains: `csp.domains`
- constraints: `csp.domains`
- arcs: `csp.arcs`

#### Example

$$\mathrm{variables} = \{A, B\},$$
$$\mathrm{domains} = \{A\in\{1,2,3\},B\in\{1,2,3\}\},$$
$$\mathrm{constraints} = \{f_0, f_1, f_2\}$$

where $f_0: A \mapsto A >= 2$, $f_1: A,B \mapsto A\neq B$, and $f_2: B\mapsto B\text{ is even}$


In [ ]:
A = Variable({1,2,3}, name='A')  # variable A and its domain
B = Variable({1,2,3}, name='B')  # variable B and its domain

In [ ]:
constraints = {
    Factor(lambda a: a>=2, A),
    Factor(lambda a,b: a != b, (A,B)),
    Factor(lambda b: b%2 == 0, B)
}

In [ ]:
csp = ConstraintSatisfactionProblem([A,B], constraints)

In [ ]:
csp.variables

In [ ]:
csp.domains

In [ ]:
csp.constraints

In [ ]:
csp.is_consistent

In [ ]:
csp.is_complete

## Node Consistency

A graph is node consistent when all values in a variable's domain satisfy that unary constraints.

In [ ]:
A = Variable({1,2,3}, name='A')  # variable A and its domain
B = Variable({1,2,3}, name='B')  # variable B and its domain

constraints = {
    Factor(lambda a: a>=2, A),
    Factor(lambda a,b: a != b, (A,B)),
    Factor(lambda b: b%2 == 0, B)
}

csp = ConstraintSatisfactionProblem([A,B], constraints)
csp.domains

We can check this by iterating through the variables and checking each possible value of the variable. For any unary constraints on that variable, if the value does not satisfy them, we should remove it from the domain.

You can see this in action in the following code:

In [ ]:
def make_node_consistent(csp):
    for variable in csp.variables:  # for eah variable in csp
        if variable.is_assigned: continue  # ignore any assign
            
        for value in variable.domain.copy(): # copy() to avoid set size change errors 
            variable.value = value  # temporarily assign value
            for constraint in csp.constraints:  # for all constraints
                if constraint.is_unary and variable in constraint:  # that are unary on variable
                    if not constraint.is_satisfied:  # if invalid
                        variable.domain.remove(value)  # remove value from domain
                        break  # no point carrying on
            variable.value = None  # unassign value and move to next

We can execute it and see that we get the same result as in the lecture.

In [ ]:
make_node_consistent(csp)
csp.domains

In [ ]:
csp.arcs

## Arc Consistency

A csp is arc consistent when all values in a variabl'e's domain satisfy that variable's binary constraint.

An arc is directional interpretation of an edge in a factor graph.

An arc can only be defined from  binary constraint: each binary constraint has two arcs.

For a factor $f(A,B)$ we have the arc such that makes $A$ to $B$ and $B$ to $A$, such that $f(A,B) \equiv\hat{f}(A,B) \equiv \bar{f}(B,A)$.

For example, for constraint, $f: A,B \mapsto A > B$ we have two arcs: $A > B$ and $B < A$. This allows is the check the values of the first against the values of the second.

To calculate arc consistency, we use an algorithm called AC-3. This has two main functions, the core AC-3 loop and a _revise_ function that takes an factor arc, $A\to B$ and checks that all values in the domain of $A$ have a value in the domain of $B$ that can satisfy the constraint.

If any value for $A$ does not have a permissible value for $B$, it is removed from the domain. We mark that the factor has been changed, as the AC-3 will need to perform more checks.

```python
def revise(factor, A, B):
    is_revised = False
    for value in A.domain:
        A.value = value
        is_valid_B = False
        for _value in B.domain:
            B.value = _value
            if factor.is_satisfied:
                is_valid_B = True
            B.value = None
        if not is_valid_B:
            A.domain.remove(value)
            is_revised = True
        A.value = None
    return is_revised
```

The AC-3 loop then uses this _revise_ function to iterate through arcs and revise. However if the domain is changed for any variable, any arcs that map to the revised variable are added back to the queue of arcs for potential revision.

AC-3 is successful once it has revised all arcs, unless there is ever a domain that is empty: in which case there is no valid assignment for this CSP.

```python
def ac3(csp):
    arcs = queue(csp.arcs)
    while len(arcs) > 0:
        f, A, B = arcs.pop()
        if revise(f, A, B):
            if len(A.domain) == 0:
                return False
            for factor in csp.constraints:
                if factor.is_binary and A in factor and B not in factor:
                    for arc in factor.arcs:
                        if arc[-1] is A:
                            arcs.push(arc)
    return True
```

Continuing with our function:

In [ ]:
csp.domains

In [ ]:
from constraints.csp.util import ac3
ac3(csp)

In [ ]:
csp.domains

### Worked Example

$$\mathrm{variables} = \{A, B\},$$
$$\mathrm{domains} = \{A\in\{1,2,3\},B\in\{1,2,3\}, C\in\{1,2,3\}\},$$
$$\mathrm{constraints} = \{f_0, f_1\}$$

where 

$f_0: A,B \mapsto A > B$

$f_1: B,C \mapsto B=C$

In [ ]:
X, Y, Z = (Variable({1,2,3}, name) for name in ("A","B","C"))

print(f"{X.name}: {X}, {Y.name}: {Y}, {Z.name}: {Z}")

constraints = [
    Factor(lambda a,b: a>b, (X,Y)),
    Factor(lambda a,b: a==b, (Y,Z))
]

csp = ConstraintSatisfactionProblem([X,Y,Z], constraints)
print(csp.variables)
print(csp.domains)
print(csp.constraints)

In [ ]:
from constraints.csp.util import ac3
ac3(csp)

In [ ]:
print(csp.domains)